In [78]:
# This code is response for generating the reg_map for the CIM_Core Module
# the responsable xslx is in the “CVA6 SoC 集成” dir


In [79]:
import argparse
from enum import Enum
from math import ceil, log


In [80]:
class Access(Enum):
  RO = 1
  RW = 2

In [81]:
class AddrMapEntry(object):
    def __init__(self, addr, name, description, access, width):
        super(AddrMapEntry, self).__init__()
        self.addr = addr
        self.description = description
        self.access = access
        self.width = width
        self.name = name

    def __str__(self):
        return '{} | {} | {} | {}'.format(hex(self.addr), self.width, self.access, self.description)

    def get_list_elem(self):
        return [hex(self.addr), self.width, self.access, self.description]


In [103]:
class AddrMap:
    def __init__(self, name, description, access_width=32):
        self.access_width = access_width
        self.name = name
        self.description = description
        self.addrmap = []
        self.ports = []

    def addEntries(self, num, addr, name, description, access, width):
        # register port
        self.ports.append((name, num, width, access))
        for i in range(0, num):
            effect_addr = addr
            # we need to split the entry into multiple aligned entries as otherwise we would
            # violate the access_width constraints
            if (width / self.access_width) > 1.0:
                for i in range(0, int(ceil(width / self.access_width))):
                    if (width - self.access_width * i < self.access_width):
                        self.addrmap.append(AddrMapEntry(effect_addr + int(self.access_width/8) * i,  name, description.format(i), access, width - self.access_width * i))
                    else:
                        self.addrmap.append(AddrMapEntry(effect_addr + int(self.access_width/8) * i,  name, description.format(i), access, self.access_width))
            else:
                self.addrmap.append(AddrMapEntry(effect_addr, name, description.format(i), access, width))

    def addEntry(self, addr, name, description, access, width):
        self.addEntries(1, addr, name, description, access, width)
        
    """Dump Verilog"""
    def emit_verilog(self):
        output = "// Do not edit - auto-generated\n"
        params =",\n".join(map(lambda x: "  " + x, [
            "parameter type reg_req_t  = logic",
            "parameter type reg_rsp_t  = logic"]))
        output += "module {} {}(\n".format(self.name, "#(\n{}\n)".format(params))
        for i in self.ports:
            # self.ports.append((name, num, width, access))
            if i[3] == str(Access.RO):
                output += "  input logic [{}:0][{}:0] {}_i,\n".format(i[1]-1, i[2]-1, i[0])
                output += "  output logic [{}:0] {}_re_o,\n".format(i[1]-1, i[0])
            elif i[3] == str(Access.RW):
                output += "  input logic [{}:0][{}:0] {}_i,\n".format(i[1]-1, i[2]-1, i[0])
                output += "  output logic [{}:0][{}:0] {}_o,\n".format(i[1]-1, i[2]-1, i[0])
                output += "  output logic [{}:0] {}_we_o,\n".format(i[1]-1, i[0])
                output += "  output logic [{}:0] {}_re_o,\n".format(i[1]-1, i[0])
            else:
                output += "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,\n"
                
        output += "  // Bus Interface\n"
        output += "  input  reg_req_t req_i,\n"
        output += "  output reg_rsp_t resp_o\n"
        output += ");\n"
        output += "always_comb begin\n"
        output += "  resp_o.ready = 1'b1;\n"
        output += "  resp_o.rdata = '0;\n"
        output += "  resp_o.error = '0;\n"
        for i in self.ports:
            if i[3] != str(Access.RO):
                output += "  {}_o = '0;\n".format(i[0])
                output += "  {}_we_o = '0;\n".format(i[0])
                output += "  {}_re_o = '0;\n".format(i[0])
        output += "  if (req_i.valid) begin\n"
        output += "    if (req_i.write) begin\n"
        output += "      unique case(req_i.addr)\n"
        j = 0
        last_name = ""
        for i in self.addrmap:
            if i.access != str(Access.RO):
                if last_name != i.name:
                    j = 0
                output += "        {}'h{}: begin\n".format(self.access_width, hex(i.addr)[2:])
                output += "          {}_o[{}][{}:0] = req_i.wdata[{}:0];\n".format(i.name, j, i.width - 1, i.width - 1)
                output += "          {}_we_o[{}] = 1'b1;\n".format(i.name, j)
                output += "        end\n"
                j += 1
                last_name = i.name
        output += "        default: resp_o.error = 1'b1;\n"
        output += "      endcase\n"
        output += "    end else begin\n"
        output += "      unique case(req_i.addr)\n"
        j = 0
        last_name = ""
        for i in self.addrmap:
            if last_name != i.name:
                j = 0
            output += "        {}'h{}: begin\n".format(self.access_width, hex(i.addr)[2:])
            output += "          resp_o.rdata[{}:0] = {}_i[{}][{}:0];\n".format(i.width - 1, i.name, j, i.width - 1)
            output += "          {}_re_o[{}] = 1'b1;\n".format(i.name, j)
            output += "        end\n"
            j += 1
            last_name = i.name
        output += "        default: resp_o.error = 1'b1;\n"
        output += "      endcase\n"
        output += "    end\n"
        output += "  end\n"
        output += "end\n"
        output += "endmodule\n"
        return output

In [104]:
from openpyxl import load_workbook
def get_address_map_from_excel(file_name):
    # 加载 Excel 文件
    workbook = load_workbook(filename=file_name)

    # 获取活动工作表（第一个工作表）
    sheet = workbook.active

    data = []  # 存储每一行的数据

    # 获取列标题
    header = [cell.value for cell in sheet[1]]

    # 遍历除列标题外的每一行
    for row in sheet.iter_rows(min_row=2, values_only=True):
        # 将每一行的单元格值与列标题对应存入字典
        row_data = {header[i]: value for i, value in enumerate(row)}
        data.append(row_data)

    return data

In [105]:
def generate_verilog_from_verilog():
    # Parse the command line arguments.
    # 我们需要提供的顶层参数包括：XXXXXX
#     parser = argparse.ArgumentParser()
#     parser.add_argument("-t", "--nr_targets", metavar="NrTargets", help="number of targets (default 2)", default=2)
#     parser.add_argument("-s", "--nr_sources", metavar="NrSources", help="number of sources (default 30)", default=30)
#     parser.add_argument("-p", "--max_priority", metavar="MaxPriority", help="maximum number of priority (default 7)", default=7)
#     args = parser.parse_args()
    
#     CIM_Core_base = 0x20000000
    addrmap = AddrMap("CIM_Core_regs", "CIM_Core Address Map")

    reg_map_data = get_address_map_from_excel(r"C:\Users\yiqi jing\Desktop\CVA6 SoC 集成\CIM_Core_address_map.xlsx")

    for i in range(len(reg_map_data)):
        reg_map_append_data = reg_map_data[i]
        addrmap.addEntries(reg_map_append_data['num'], int(reg_map_append_data['addr'],16), reg_map_append_data['name'], \
                           reg_map_append_data['description'], reg_map_append_data['access'], reg_map_append_data['width'] )
    
    print(addrmap.emit_verilog())
    return

In [106]:
generate_verilog_from_verilog()

// Do not edit - auto-generated
module CIM_Core_regs #(
  parameter type reg_req_t  = logic,
  parameter type reg_rsp_t  = logic
)(
  input logic [0:0][3:0] test_1_i,
  output logic [0:0][3:0] test_1_o,
  output logic [0:0] test_1_we_o,
  output logic [0:0] test_1_re_o,
  input logic [0:0][15:0] test_2_i,
  output logic [0:0][15:0] test_2_o,
  output logic [0:0] test_2_we_o,
  output logic [0:0] test_2_re_o,
  input logic [0:0][3:0] test_3_i,
  output logic [0:0][3:0] test_3_o,
  output logic [0:0] test_3_we_o,
  output logic [0:0] test_3_re_o,
  input logic [0:0][23:0] test_4_i,
  output logic [0:0][23:0] test_4_o,
  output logic [0:0] test_4_we_o,
  output logic [0:0] test_4_re_o,
  // Bus Interface
  input  reg_req_t req_i,
  output reg_rsp_t resp_o
);
always_comb begin
  resp_o.ready = 1'b1;
  resp_o.rdata = '0;
  resp_o.error = '0;
  test_1_o = '0;
  test_1_we_o = '0;
  test_1_re_o = '0;
  test_2_o = '0;
  test_2_we_o = '0;
  test_2_re_o = '0;
  test_3_o = '0;
  test_3_we_o = '0;


In [107]:
import os

current_path = os.getcwd()
print("当前路径：", current_path)

当前路径： C:\Users\yiqi jing\CVA6_python
